<a href="https://colab.research.google.com/github/thad75/TP_ENSEA_ELEVE/blob/main/SIA/TP2/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoders

Time : 8h

# Preparatory Questions 

## General Questions
- First watch this video : https://www.youtube.com/watch?v=Rdpbnd0pCiI
- What is the difference between supervised and unsupervised learning ?
- What does it mean to train a model for a certain task ?
- What is a Loss function ?

## AutoEncoder questions
- Is an AutoEncoder a Supervised or Unsupervised algorithm in the case of Image Compression
- What loss function should be used in an AutoEncoder ?
- Go on the following link : https://cs.stanford.edu/people/karpathy/convnetjs/demo/autoencoder.html. What's the purpose of this website ? 
- Play a lil' bit with that website. Just above the "Network Visualization" Title, you have a drawing of digits (0,1,2.....9). Explain what that visualisation is ? 
- Do you see clusters ? Is it good or no ?

# Disclaimer

We will refer to :
- AutoEncoder as AE
- Multi Layer Perceptron as MLP


# Introduction

So basically during the part 1 of this lab, we have seen JPEG Compression. JPEG compression is a general algorithm that can compress any image. Let's first see your understanding of the JPEG algorithm :
* What are the component in the Encoding part of the algorithm ?
* What are the component in the Decoding part of the algorithm ?
* Is the down-sampling phase of JPEG linear ? (i.e : in a y = ax+b form)
* Is it a lossless compression algorithm ?

# AI for the Win

Now let's do some Deep Learning for Image compression. This might be your first time using Deep Learning, so don't be afraid we will guide you.

Goal of this lab :
* Get to know Deep Learning
* Learn how to compress images using AutoEncoders
* Understand the differences between JPEG compression and Deep Learning for compression

Alright, let's get started.

<img src="https://i.pinimg.com/originals/16/b2/96/16b296afb78ec57d12c931bc72b42eec.gif">

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms

# What is Deep Learning ?

Deep Learning is a branch of AI where you **teach a Model** a certain **task** using a **Dataset**. The Model is composed of **Layers of Neurons** that are updated using a **Loss**. The Model infers a prediction from an **input**. In fact, a Deep Neural Network can be seen as a complex function ${f}$ that maps the input data to a learned space from the Dataset. Note the bold words. These are the important things you need to understand about Deep Learning. 

# Generalities on AutoEncoder

An autoencoder is a model able to reconstruct an input from a Latent Space.
An autoencoder (AE) is composed of 2 models :
* an encoder model that encodes input data into a latent space
* a decoder model that recreates the input image from the latent space



Think of it as the Chinese Whispers (Téléphone Arabe en francais) game :
- First player says a sentence
- Second player repeats the sentence of First player to Third player
- Third player repeats the sentence of Second player to Fourth player
- ...
- The final player repeats the sentence of Before Last Player, which by recurrence should be what First Player said.

This basically is an AutoEncoder :
- Player 1 sends a Voice signal that is compressed by all the Players
- Last Player must reconstruct perfectly the Voice Signal of Player 1 from the compressed signal of all the previous players.


<img src = "https://journals.openedition.org/marges/docannexe/image/364/img-2.jpg" width="400" height="500">

You will code two types of AE models :
* MLP style
* Conv style


# Data

## Dataset
When training  a DL model, we use a Dataset. The model uses the Dataset to learn something for a task. We usually divide the Dataset into Train, Valid, Test Dataset.
In this lab, we will use MNIST Dataset. First, let's see the Dataset. We do not have to code the Dataset as Folks from Torchvision wrote it for us.

- Load the Dataset from TorchVision

In [ ]:
# TODO: Load MNIST Train Dataset from TorchVision

dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())

# TODO: What's the size of the Dataset ?
# TODO: Retrieve one element of the Dataset ? What is the shape of one piece of Data ? 

size_of_dataset =  len(dataset)
data = dataset[0]
print(data.shape)

# TODO: Plot the retrieved Data

plt.imshow(data.permute(1, 2, 0))

As you can see, there's a train attribute to the MNIST Class. When it's set to True, you're loading the train Dataset. Hence, change it to false to load the test Dataset

In [ ]:
mnist_test =  MNIST('', train=False, download=True, transform=transforms.ToTensor())



We will create a Validation Dataset.

In [ ]:
mnist_train, _ = 

In [ ]:
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_test =  MNIST('', train=False, download=True, transform=transforms.ToTensor())

mnist_train, _ = 

train_loader = 


## Dataloader

So the Dataset returns one element at a time. In DL, we like sending many items at the same time to the model. We form BATCH of Data and send it using a DataLoader. Dataloader are an iterable over the dataset. It means that the Dataloader will form BATCH of Data for you and fetch them when training.
- Create a DataLoader for your Training, Valid and Testing Dataset

# Creating the Neural Nets

## MLP Style

<img src='https://www.researchgate.net/publication/344394387/figure/fig1/AS:974657746399232@1609387923440/Figure-Computational-Schematics-of-the-MLP-and-the-autoencoder.png'>

The first type of AutoEncoder, you will code a MLP style AE. In this part, you will :
- Create Layers that inherits from nn.Module

The model is composed of an input layer, a bottleneck, and an output layer.
- Where do you think the compression occurs ?

The bottleneck forces a compressed representation of the original input. 

You will create few nets :
- MLPDown that compresses the input Image
- Encoder that stacks multiples MLPDown to create the Latent Space
- MLPUp that uses the Latent Space to decode and recreate the Input Image
- Decoder that stack multiple MLPUp to recreate the Compressed Input Image.

Quick Tips : A PyTorch module is usually composed of two methods : 

- Init to initialize the class
- forward to forward the data through your model

In [ ]:
class MLPDown(nn.Module):

    def __init__(self, input_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.model = nn.Sequential(nn.Linear(self.input_size, self.output_size),
                                    nn.ReLU())
                                    
    def forward(self,x):
        return self.model(x)

In [ ]:
class MLPUp(nn.Module):

    def __init__(self, input_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.model = nn.Sequential(nn.Linear(self.input_size, self.output_size),
                                    nn.ReLU())
                                    
    def forward(self,x):
        return self.model(x)

In [ ]:
class Encoder(nn.Module):

    def __init__(self,input_size, latent_size):
        super().__init__()
        self.input_size = input_size
        self.latent_size = latent_size
        self.model = nn.Sequential( 
                        MLPDown(self.input_size, self.latent_size))
                    
    def forward(self,x):
        return self.model(x)

In [ ]:
class Decoder(nn.Module):

    def __init__(self, latent_size,input_size):
        super().__init__()
        self.input_size = input_size
        self.latent_size = latent_size
        self.model = nn.Sequential( 
                        MLPUp( self.latent_size,self.input_size))
                    
    def forward(self,x):
        return self.model(x)

In [ ]:
class AutoEncoder(nn.Module):

    def __init__(self,input_size,latent_size):
        super().__init__()
        self.input_size = input_size
        self.latent_size = latent_size
        self.model = nn.Sequential(Encoder(self.input_size,self.latent_size),
                                   Decoder(self.latent_size, self.input_size))

    def forward(self,x):
        return self.model(x)

## Encoder

The encoder encodes the input image(s). Indeed, it takes the image and extracts features thanks to its different layers. The deeper the model, the complexer the extracted features will be. At the end of the Decoded Network you will have a latent representation of the image.
* However is it necessary to have very Deep Networks ? 
* Create a **Fully Connected** Encoder Model. 

Your model must be instanciated using two parameters : 
* Input shape
* Latent size

What are those parameters ?

Feel free to design your own model, by using whatever Activation function and Layer size you want (but can you ?)

# Dataset

You will use the MNIST dataset in order to train and test you model. You won't have to code the Dataset as Pytorch has already coded it for you. Please refer to the introduction to Lightning if needed.

Load the MNIST dataset, and setup what is needed for the training. Refer to the 1rst lab if needed.


In [ ]:
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_test =  MNIST('', train=False, download=True, transform=transforms.ToTensor())

mnist_train, _ = 

train_loader = 


# Autoencoder


# Encoder

The encoder encodes the input image(s). Indeed, it takes the image and extracts features thanks to its different layers. The deeper the model, the complexer the extracted features will be. At the end of the Decoded Network you will have a latent representation of the image.
* However is it necessary to have very Deep Networks ? 
* Create a **Fully Connected** Encoder Model. 

Your model must be instanciated using two parameters : 
* Input shape
* Latent size

What are those parameters ?

Feel free to design your own model, by using whatever Activation function and Layer size you want (but can you ?)

In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_shape,latent_size):
        super().__init__()
        
    def forward(self,x):
        return x

# Decoder


The Decoder takes as input the latent representation of the input vector and recreates the input image.
* Create a Decoder model.

Your model must be instanciated using two parameters : 
* Output shape
* Latent size

What are those parameters ? Are they different from the Encoder ?


Feel free to design your own model, by using whatever Activation function and Layer size you want (but can you ?)


In [ ]:
class Decoder(nn.Module):
  def __init__(self, latent_size, output_shape):
        super().__init__()
        
  def forward(self,x):
      return x


# Autoencoder


Let's create the AutoEncoder. Using Lightning Framework,  :     
* Stack the encoder and the decoder in order to create the AE. 

Now let's think a little bit before beginning :
* What task is it ? (Classification ? Regression ?)
* What optimizer will you be using ?
* What learning rate ?
* Will your model learn ? (overfit_batches will help)
* What is the impact of the latent_space size ?
* What loss shold we be using ?

Don't forget to log the needed values 

In [ ]:
class AutoEncoder(pl.LightningModule):
    def __init__(self, latent_size, input_shape, output_shape):
        super(AutoEncoder,self).__init__()

        self.encoder = 
        self.decoder = 
        self.save_hyperparameters()

    def forward(self,x):
        reconstructed_image = 
        return reconstructed_image

    def configure_optimizers(self):
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x,y = train_batch
        return loss


    def validation_step(self, val_batch, batch_idx):
        x,y = val_batch
        self.log('val_loss', loss)

Set up Tensorboard for observation. The given directory is the default directory and might not the created at the moment you run the code. In fact, the trainer will create the folder where the checkpoints are located.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/tb_logs/my_model/version_0"

Now let's create your Trainer. 
* Load your model
* Create your lightning Trainer
* Fit the data to your model.

Once your trainer set up, train for 10 epochs and watch your training.

In [ ]:
model = AutoEncoder(_, _)
trainer = pl.Trainer(gpus=-1,max_epochs=_)
trainer.fit(model, train_loader, val_loader)

# Test


Now that you have trained your model, let's test it. We provide you a function that will help you to load your model and use it. 
You might need to do something on your testing data before sending it to the model. What is it ?

Try few images reconstructions.
* Where should you pick your testing data from ? 
* Can we test on training set ? Why ?


In [ ]:
import os 
from os import listdir
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
def load_model_from_checkpoint(model, checkpoint_path):
    ckp = os.listdir(checkpoint_path+'/checkpoints/')
    ckp = checkpoint_path + '/checkpoints/' + ckp[0]
    model = model.load_from_checkpoint(ckp)
    model.freeze()
    return model

def forward_and_print_image(model,image):
  image = model(test_image.view(1,-1))
  image_reco = image.view(28,28)
  plt.imshow(image_reco)


model = load_model_from_checkpoint(model,_)
test_image = mnist_val.__getitem__(1)[0].unsqueeze(0)
forward_and_print_image(model,test_image)

# Convolutional Type

In the previous episode of AutoEncoder, we have seen Fully Connected AE. Let's see another one : Convolutional AE.

We will make you code in a different way, using Module Lists. But at the end the results will be the same : we are going to make the same reconstruction using Convolution Layers.

By looking at the following links, **explain each parameters of the nn.Conv2d and nn.ConvTranspose2d methods** :
* https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html
* https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
* https://d2l.ai/chapter_convolutional-neural-networks/padding-and-strides.html
* https://ezyang.github.io/convolution-visualizer/


We will ask you few things :
* Calculate the size of the images after each pass through a layer of your model. Write the formula, you're using for ths calculation.
* Are you going to use MaxPooling ? If yes, explain with a photo what Maxpooling does. 
* Can we revert MaxPooling effect without information loss ?

You will still use the MNIST Dataset

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

# Encoder Conv

First let's write one Bottleneck of your Convolutional Encoder.

Your Model should be instanciated using :    
* in_channels : number of input channels
* out_channels : number of output channels

Why ?


In [ ]:
class ConvDown(nn.Module):
      def __init__(self, in_channels, out_channels):
            super(ConvDown,self).__init__()
            self.conv = nn.Sequential(
                nn.Conv2d(...

      def forward(self,x):

          return x

# Decoder Conv

blablabla.
Write your Decoding Bottleneck.
Be careful if you used MaxPooling for the shape of your inputs. Don't hesitate to print each output shape. 


Your Model should be instanciated using :    
* in_channels : number of input channels
* out_channels : number of output channels

In [ ]:
class ConvUp(nn.Module):
      def __init__(self, in_channels, out_channels):
            super(ConvUp,self).__init__()
            self.conv = nn.Sequential(
                nn.ConvTranspose2d(....
            )

      def forward(self,x):
          return x



# Create your Conv AE.


Let's make things a Lil' bit spicy. We are going to stack the Bottlenecks in order to create our Encoder and Decoder. Then, we are going to stack the encoder and the Decoder in order to create the AutoEncoder. Easy ?

Your model must be instanciated using :
* in channels 
* out_channels
* feature channel shape : list of feature channel shape you want.


The given feature_shape can be indeed changed.
Using nn.ModuleList() create your encoder and your decoder. 
Have a look at : https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html


We have written the forward for you. Take some inspiration from it to define your training step. Don't forget to log some values.




In [ ]:
class AutoEncoderConv(pl.LightningModule):
    def __init__(self, in_channels, out_channels,feature_shape = [3,16,32]):
        super(AutoEncoderConv,self).__init__()
        self.save_hyperparameters() #mandatory
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.encoder = nn.ModuleList()
        self.decoder = nn.ModuleList()
        self.feature_shape = feature_shape
        self.reverse_list = list(reversed([self.out_channels]+self.feature_shape))
        for feature in self.feature_shape:
            self.encoder.append(_) #UP or DOWN ?
            in_channels = feature
        for i,_ in enumerate(self.reverse_list):
            if i < len(self.reverse_list) -1:
                self.decoder.append(_) #UP or DOWN ?


    def forward(self,x):
        for i,down in enumerate(self.encoder):
          x = down(x)
        x_reconstructed=x
        for i,up in enumerate(self.decoder):
          x_reconstructed = up(x_reconstructed)
        return x_reconstructed

    def configure_optimizers(self):
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x,y = train_batch
        return loss


    def validation_step(self, val_batch, batch_idx):
        x,y = val_batch
        self.log('val_loss', loss)



# Load your Logger

When training a model you want to get a log of every useful values.

In [ ]:
logger = TensorBoardLogger("tb_logs", name="my_model")

# Train

Fit that model. Don't forget to load tensorboard before training

In [ ]:
model = AutoEncoderConv(_, _)
trainer = pl.Trainer(gpus=-1,logger = logger,max_epochs = _)
trainer.fit(model, train_loader, val_loader)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/tb_logs/my_model/version_0"

# Test

Now by loading the last checkpoint of your trained model, show some reconstructed images from the Test set.

* How is the reconstruction ?

We provide two functions to help you in this mission :
* load_model_from_chekcpoint that return the model with the weights loaded
* forward_and_print_image that forwards the image through the model and prints the output

Don't forget these squeezes...

In [ ]:
import os 
import pickle
from os import listdir
import matplotlib.pyplot as plt

def load_model_from_chekcpoint(model, checkpoint_path):
    ckp = os.listdir(checkpoint_path+'/checkpoints/')
    ckp = checkpoint_path + '/checkpoints/' + ckp[0]
    checkpoint = torch.load(ckp)  
    model =  model.load_from_checkpoint(ckp,hparams_file = checkpoint_path+'/hparams.yaml')
    model.freeze()
    return model

def forward_and_print_image(model,image):
  image = model(test_image)
  image_reco = image.view(28,28)
  plt.imshow(image_reco)


model = load_model_from_chekcpoint(_,_)
test_image = 
plt.imshow()


In [ ]:
forward_and_print_image(_,_)

AutoEncoders got few advantages :
* It compresses images 
* It is used for denoising Images
* It can be used for Anomaly Detection.

# Adding Classification

Let's get back to FC AE.
Let's classify something. Now we are going to classify the reconstructed image.
* What task are we adding to the model ? How can we add it ?
* Where should we add the corresponding layer ? Draw a Schema of your model. What's the name of these type of models ?
* What loss should we use ?
* Can we calculate an accuracy ? 

In [ ]:
class AutoEncoder_for_Accuracy(pl.LightningModule):
    def __init__(self, latent_size, input_shape,num_class):
        super(AutoEncoder_for_Accuracy,self).__init__()
        self.latent_size = latent_size
        self.input_shape = input_shape
        self.num_class = 
        self.encoder = 
        self.decoder = 
        self.fc = 
        self.save_hyperparameters()

    def forward(self,x):
        
        return reconstructed_image,label_hat

    def configure_optimizers(self):
        
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x,y = train_batch
       
        self.log('train_loss', loss)
        self.log('train_acc', accuracy)

        return loss+loss_cls


    def validation_step(self, val_batch, batch_idx):
        x,y = val_batch
        
        self.log('val_loss', loss)
        self.log('val_acc', accuracy)



    def test_step(self, test_batch, batch_idx):
        x,y = test_batch
        
        self.log('test_loss', loss)
        self.log('test_acc', accuracy)

Train your model

In [ ]:
logger = TensorBoardLogger("tb_logs", name="my_model")
model = AutoEncoder_for_Accuracy(_, _,_)
trainer = pl.Trainer(gpus=-1,logger = logger,max_epochs=10)
trainer.fit(model, train_loader, val_loader)

# Load and Test

In [ ]:
import os 
import pickle
from os import listdir
import matplotlib.pyplot as plt

def load_model_from_chekcpoint(model, checkpoint_path):
    ckp = os.listdir(checkpoint_path+'/checkpoints/')
    ckp = checkpoint_path + '/checkpoints/' + ckp[0]
    checkpoint = torch.load(ckp)  
    model =  model.load_from_checkpoint(ckp,hparams_file = checkpoint_path+'/hparams.yaml')
    model.freeze()
    return model

def forward_and_print_image(model,image):
    image,label = model(test_image)
    label = torch.argmax(label, dim =1)
    print('label is: ', label.detach().cpu().numpy()[0])
    image_reco = image.view(28,28)
    plt.imshow(image_reco)


model = load_model_from_chekcpoint(model,"/content/tb_logs/my_model/version_1/")
test_image = mnist_test.__getitem__(1)[0].unsqueeze(0)
plt.imshow(test_image.squeeze(0).squeeze(0))
forward_and_print_image(model,test_image.view(28,28)) ####reconstruit

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/tb_logs/my_model/version_1"

# Denoising Images


AutoEncoder has that good ability to denoise your input image. Let's try it !
* Add various noises to your input images and test what your AutoEncoder outputs.
* Where should you pick your test images from ?

In [ ]:
def add_noise(inputs):
     noise = _
     return inputs + noise

In [ ]:
model = load_model_from_chekcpoint(_,_)
test_image = 
test_image_with_noise = 
plt.imshow()

In [ ]:
forward_and_print_image(_,_)

Now let's use the AutoEncoder for harder tasks.